In [129]:
import os

import cv2
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from dask import delayed


In [134]:
from sklearn.linear_model import Perceptron

##  Integral Image

Computes rectangle integrals of an image for fast computation of Harr Filters.

In [2]:
def compute_intg_image(img):
    """
    Computes matrix for quick computation of integrals.
    """
    # Create a matrix of zeros with the same dimensions as the input image
    intg_img = np.zeros(img.shape, dtype=np.int64)
    
    # Iterate over each pixel in the input image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the sum of all pixels above and to the left of the current pixel
            if i > 0:
                intg_img[i, j] += intg_img[i-1, j]
            if j > 0:
                intg_img[i, j] += intg_img[i, j-1]
            if i > 0 and j > 0:
                intg_img[i, j] -= intg_img[i-1, j-1]
            
            # Add the value of the current pixel to the integral image
            intg_img[i, j] += img[i, j]
    
    # Return the computed integral image
    return intg_img

def get_rect(intg_img, si, sj, ei, ej):
    # Calculate the sum of all pixels within the specified rectangular region
    result = intg_img[ei, ej]
    if si > 0:
        result -= intg_img[si-1, ej]
    if sj > 0:
        result -= intg_img[ei, sj-1]
    if si > 0 and sj > 0:
        result += intg_img[si-1, sj-1]
    
    # Return the computed sum of pixels
    return result

In [ ]:
img = np.ones((3,3))
intg_img = compute_intg_image(img)

In [7]:
list(range(0,3,1))

[0, 1, 2]

In [67]:
get_rect(intg_img, 0,0, 2,1)

6.0

In [41]:
face_imgs = []
for file in os.listdir('data/'):
    if ".pgm" in file:
        face_imgs.append(cv2.imread("data/"+file,-1))
cv2.imshow("face", np.hstack(face_imgs))
cv2.waitKey(0)

113

In [94]:
def load_folder(folder):
    imgs = []
    for file in os.listdir(folder):
        if ".pgm" in file:
            imgs.append(cv2.imread(folder+"/"+file,-1))
    return imgs

In [77]:
face_imgs[0].shape

(120, 128)

In [27]:
from skimage.feature import haar_like_feature, draw_haar_like_feature, haar_like_feature_coord

In [16]:
haar_like_feature(compute_intg_image(face_imgs[0]),0,0,face_imgs[0].shape[1], face_imgs[0].shape[0])

In [29]:
draw_haar_like_feature(intg_img,0,0,19, 19, haar_like_feature_coord())

TypeError: draw_haar_like_feature() missing 1 required positional argument: 'feature_coord'

In [76]:
intg_img = compute_intg_image(face_imgs[0])

In [77]:
intg_img.shape

(19, 19)

In [40]:
img = face_imgs[0]
kernel_size = 40
for i in range(0, img.shape[0], 10):
    for j in range(0, img.shape[1], 10):
        if i+kernel_size <= img.shape[0] and j+kernel_size <= img.shape[1]:
            cv2.imshow("img",cv2.rectangle(img.copy(), (j, i), (j+kernel_size-1, i+kernel_size-1), 255, 2 ))
            cv2.imshow("roi", cv2.resize(img[i:i+kernel_size, j:j+kernel_size], (100,100)))
            cv2.waitKey(100)

In [117]:
def compute_haar_features(intg_img, haar_filters, feature_size):
    haar_features = np.zeros(feature_size)
    idx = 0
    for filter in haar_filters:
        for i in range(intg_img.shape[0]-filter[0]+1):
            for j in range(intg_img.shape[1]-filter[1]+1):
             
                if filter[2]:
                    result = ( get_rect(intg_img, i, j, i+filter[0]-1, j+filter[1]//2-1) 
                    - get_rect(intg_img, i, j+filter[1]//2, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])
                else:
                    result = ( get_rect(intg_img, i, j, i+filter[0]//2-1, j +filter[1]-1) 
                    - get_rect(intg_img, i+filter[0]//2, j, i+filter[0]-1, j+filter[1]-1) )/((filter[0])*filter[1])
                haar_features[idx] = result
                idx += 1
            
        
    return haar_features
def generate_haar_filters(detector_size=19):
    haar_feature_size = 0
    # (r, c, is_vert)
    haar_filters= []
    for i in range(1, detector_size+1):
        for j in range(1, detector_size+1):
            if i % 2 == 0:
                haar_filters.append((i,j, 0))
                haar_feature_size += (19-i+1) * (19-j+1)
            if j % 2 == 0:
                haar_filters.append((i,j, 1))
                haar_feature_size += (19-i+1) * (19-j+1)
    return haar_feature_size, haar_filters

def get_haar_feature_extractor(detector_size):
    haar_feature_size, haar_filters = generate_haar_filters(detector_size)
    @delayed
    def get_haar(intg_img):
        return compute_haar_features(intg_img, haar_filters, haar_feature_size)
    return get_haar

haar_feature_extractor = get_haar_feature_extractor(19)


array([ -6.5       ,  -8.5       , -11.        , ...,   0.96381579,
        -5.00292398,  -0.47076023])

In [90]:
compute_haar_features(intg_img, haar_filters).shape

(34200,)

In [113]:
len(train_x_false)

4548

In [115]:
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.2 MB/s eta 0:00:00
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00:00


In [124]:
train_x_true = load_folder("train/face")[:200]
train_x_false = load_folder("train/non-face")[:200]
train_y_true = [1] * len(train_x_true)
train_y_false = [0] * len(train_x_false)

train_x = train_x_true + train_x_false
train_y = train_y_true + train_y_false

train_x = delayed(haar_feature_extractor(compute_intg_image(img)) for img in train_x)

train_x = np.array(train_x.compute(scheduler='single-threaded')))
train_y = np.array(train_y)

In [127]:
np.power(2,[1,2,3])

array([2, 4, 8])

AttributeError: module 'sklearn' has no attribute 'linear_model'

In [168]:
def Adaboost(train_x, train_y, haar_feature_extractor, T=100):
    face_count = 0
    non_face_count = 0
    
    for i, label in enumerate(train_y):
        if label:
            face_count += 1
        else:
            non_face_count += 1

    w = np.array([1/face_count if train_y[i] else 1/non_face_count for i in range(len(train_y))])

    classifiers = []
    beta = np.zeros(T)
    for t in range(T):
        w = w / np.sum(w)

        model = Perceptron()
        model.fit(train_x, train_y, w)
        e = np.abs(train_y - model.predict(train_x))
        
        beta[t] = np.sum(w * e)
        w = w * np.power(beta[t], 1-e)
        classifiers.append(model)
    alpha = np.log(1/beta)
    predictions = np.sum(np.array([c.predict(train_x) for c in classifiers]) * alpha, axis=0)[:face_count]
    threshold = np.min(predictions)
    print(threshold)
    

      

In [170]:
import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [169]:
face_count = 0
non_face_count = 0
T = 400
for i, label in enumerate(train_y):
    if label:
        face_count += 1
    else:
        non_face_count += 1
w = np.array([1/face_count if train_y[i] else 1/non_face_count for i in range(len(train_y))])

classifiers = []
beta = np.zeros(T)
for t in range(T):
    w = w / np.sum(w)

    model = Perceptron()
    model.fit(train_x, train_y, sample_weight=w)
    e = np.abs(train_y - model.predict(train_x))
    
    beta[t] = np.sum(w * e)
    w = w * np.power(beta[t], 1-e)
    classifiers.append(model)
alpha = np.log(1/beta)
predictions = np.sum( np.array([c.predict(train_x) for c in classifiers]) * alpha.reshape(-1,1), axis=0)[:face_count]
threshold = np.min(predictions)
print(threshold)
    

189.64181756943333


In [160]:
def predict(data):
    predictions = np.sum( np.array([c.predict(data) for c in classifiers]) * alpha.reshape(-1,1), axis=0)
    return predictions >= threshold

In [155]:
test_x_true = load_folder("train/face")[200:220]
test_x_false = load_folder("train/non-face")[200:220]
test_y_true = [1] * len(test_x_true)
test_y_false = [0] * len(test_x_false)

test_x = test_x_true + test_x_false
test_y = test_y_true + test_y_false

test_x = delayed(haar_feature_extractor(compute_intg_image(img)) for img in test_x)
test_x = np.array(test_x.compute(scheduler='single-threaded'))
test_y = np.array(test_y)



In [172]:
predict(test_x)

array([ True,  True,  True,  True, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True, False, False, False,  True,
       False, False,  True, False,  True, False,  True, False,  True,
        True, False, False, False])